In [2]:
import pandas as pd
import numpy as np




# pd.reset_option('^display.', silent=True)
# pd.options.display.float_format = '{:.1f}'.format
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 700)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 150)

In [3]:
import sys
sys.path

# from pandas_profiling import ProfileReport

['',
 'C:\\Users\\Yossi\\Desktop\\Python Learning\\GFK POC\\poc_ooh_batch2',
 'C:\\Users\\Yossi\\AppData\\Local\\Continuum\\anaconda3\\python37.zip',
 'C:\\Users\\Yossi\\AppData\\Local\\Continuum\\anaconda3\\DLLs',
 'C:\\Users\\Yossi\\AppData\\Local\\Continuum\\anaconda3\\lib',
 'C:\\Users\\Yossi\\AppData\\Local\\Continuum\\anaconda3',
 'C:\\Users\\Yossi\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\Users\\Yossi\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages',
 'C:\\Users\\Yossi\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\Yossi\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Yossi\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Yossi\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Yossi\\.ipython']

In [2]:
ls

 Volume in drive C has no label.
 Volume Serial Number is AAF9-F065

 Directory of C:\Users\Yossi\Desktop\Python Learning\GFK POC\poc_ooh_batch2

12/02/2020  13:17    <DIR>          .
12/02/2020  13:17    <DIR>          ..
03/02/2020  13:58    <DIR>          .ipynb_checkpoints
10/02/2020  16:59            18,260 All_retailers.png
12/02/2020  10:48            27,278 essence_review.xlsx
03/02/2020  16:08           291,140 Format_csv.xlsx
12/02/2020  10:48           178,918 GFK_POC_OOH_B2.xlsx
06/02/2020  13:24         3,338,917 hotofhome_batch2_analysis.ipynb
06/02/2020  11:29           836,240 OutOfHome_batch2.csv
06/02/2020  17:51           539,331 OutOfHome_batch2_06022020_1158.csv
09/02/2020  09:52           808,820 OutOfHome_batch2_07022020_0000.csv
10/02/2020  12:43           811,089 OutOfHome_batch2_09022020_1130.csv
10/02/2020  18:02           815,324 OutOfHome_batch2_10022020_1240.csv
11/02/2020  12:55           861,461 OutOfHome_batch2_run1.csv
11/02/2020  12:54           807,5

# Load csv file

In [81]:
df_outofhome = pd.read_csv('OutOfHome_batch2_run9.csv').sort_values(['client_receipt_id','id']).reset_index()
df_outofhome['snapshot_url_1'] = df_outofhome['snapshot_url_1'].apply(lambda x: x.split('?')[0])


## Seperate to groups by receipt's quality - from manual check file

In [82]:
df_receipts_quality = pd.read_csv('receipts_quality.csv')
df_outofhome = df_outofhome.merge(df_receipts_quality, on='client_receipt_id' , how="left")

## Remove identification for Unknown Merchants and change column names

In [83]:
df_outofhome = df_outofhome.copy()[['file',
                                    'client_receipt_id',
                                    'snapshot_url_1',
                                    'retailer_name',
                                    'purchase_date',
                                    'purchase_time',
                                    'purchase_total',
                                    'store_phone',
                                    'receipt_item_code',
                                    'receipt_item_description',
                                    'receipt_item_price',
                                    'receipt_item_quantity',
                                    'quantity_type',
                                    'identified_product_upc',
                                    'identified_product_description',
                                    'identified_product_brand',
                                    'product_category',
                                    'essence_in_item',
                                    'essence',
                                    'product_confidence',
                                    'quality',
                                   'type']]



df_outofhome.loc[df_outofhome['retailer_name'].isna(), ['identified_product_description','identified_product_upc','identified_product_brand','product_confidence']] = np.nan
df_outofhome['essence_in_item'] = df_outofhome['essence_in_item'].str.replace('\d+', '')
df_outofhome.loc[df_outofhome['essence_in_item']=='super', ['essence_in_item']] = np.nan
df_outofhome.loc[df_outofhome['essence_in_item']=='tikka', ['essence_in_item']] = 'chicken tikka'
df_outofhome.loc[df_outofhome['receipt_item_description']=='caesars salat (caesars salat', ['essence_in_item']] = 'salat'
df_outofhome.loc[df_outofhome['product_category']=='BIER', ['essence_in_item']] = 'bier'



df_outofhome.rename(columns={'identified_product_upc':'identified_gtin','identified_product_description':'identified_product_name','product_category':'identified_product_group', 'essence':'identified_product_essence'}, inplace=True)

# Save to xlsx file

In [84]:
# Seperate the data into 2 different dataframes according to receipt's quality.
df_outofhome_poor = df_outofhome[df_outofhome['quality']=='poor']
df_outofhome_ok = df_outofhome[df_outofhome['quality']=='good']


# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('GFK_POC_OOH_B2.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_outofhome_ok.to_excel(writer, sheet_name='Good_quality_receipts',index=False)
df_outofhome_poor.to_excel(writer, sheet_name='Poor_quality_receipts',index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

-----------------------------------------------------------------------------------------------------------------------

# Analysis

### Receipts missing values

In [76]:
df_outofhome_ok = df_outofhome_ok[['client_receipt_id','receipt_item_description','identified_product_name','essence_in_item', 'identified_product_essence','product_confidence','type']]
total_items_and_discounts = len(df_outofhome_ok)

df_outofhome_ok = df_outofhome_ok[df_outofhome['type']=='Item']
total_items = len(df_outofhome_ok)

df_outofhome_ok = df_outofhome_ok[~df_outofhome_ok['receipt_item_description'].str.contains('pfand',na=True, case=False, regex=False)]
total_items_without_pfand = len(df_outofhome_ok)

essence_in_item = len(df_outofhome_ok[df_outofhome_ok['essence_in_item'].isna()==False])


print('total items: ',total_items)

print('% essence_in_item from total items and discounts: ', str(round(essence_in_item/total_items_and_discounts*100)))
print('% essence_in_item from total items: ', str(round(essence_in_item/total_items*100)))
print('% essence_in_item from total items without pfands: ', str(round(essence_in_item/total_items_without_pfand*100)))

print()
print('total essence in items: ', str(essence_in_item))
print('unique essence values:',str(len(df_outofhome_ok['essence_in_item'].unique())))
print()
print('unique essence values:',str(round(len(df_outofhome_ok['essence_in_item'].unique())/essence_in_item*100)),'%')

total items:  1471
% essence_in_item from total items and discounts:  65
% essence_in_item from total items:  66
% essence_in_item from total items without pfands:  71

total essence in items:  977
unique essence values: 442

unique essence values: 45 %


C:\Users\Yossi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


# generate files for auto ml classification:


### Unified essence to gfk taxonomy

### Save txt files for each taxonomy with more than 10 item description:

# run essence comparison

In [79]:
df1 = pd.read_csv('OutOfHome_batch2_run3.csv')
df1.loc[df1['product_category']=='BIER', ['essence_in_item']] = 'BIER'
df1 = df1[['client_receipt_id','receipt_item_description','essence_in_item','identified_product_description']]
df1.loc[df1['essence_in_item']=='super', ['essence_in_item']] = ''
df1.loc[df1['essence_in_item']=='tikka', ['essence_in_item']] = 'chicken tikka'
df1.loc[df1['receipt_item_description']=='caesars salat (caesars salat', ['essence_in_item']] = 'salat'



df2 = pd.read_csv('OutOfHome_batch2_run9.csv')
df2.loc[df2['product_category']=='BIER', ['essence_in_item']] = 'BIER'
df2 = df2[['client_receipt_id','receipt_item_description','essence_in_item','identified_product_description']]
df2['essence_in_item'] = df2['essence_in_item'].str.replace('\d+', '')

df2.loc[df2['essence_in_item']=='super', ['essence_in_item']] = ''
df2.loc[df2['essence_in_item']=='tikka', ['essence_in_item']] = 'chicken tikka'
df2.loc[df2['receipt_item_description']=='caesars salat (caesars salat', ['essence_in_item']] = 'salat'






df = df1.merge(df2, on=['client_receipt_id','receipt_item_description'] ,how="inner",suffixes=('_left', '_right'))
df = df.fillna('')
df_receipts_quality = pd.read_csv('receipts_quality.csv')
df = df.merge(df_receipts_quality, on='client_receipt_id' , how="left")
df = df[df['quality']=='good']
print(len(df))
df = df[['client_receipt_id','receipt_item_description','essence_in_item_left','essence_in_item_right','identified_product_description_left','identified_product_description_right']]
df = df[df['receipt_item_description'].isna()==False]


df=df[df['essence_in_item_left']!=df['essence_in_item_right']]
# df = df[(df['essence_in_item_left']=='') & (df['essence_in_item_right']!='')]
print(len(df))

df

1592
397


,client_receipt_id,receipt_item_description,essence_in_item_left,essence_in_item_right,identified_product_description_left,identified_product_description_right
35,OHHReceipt_20200110_114028,coke zero 1 l pet,,cola,"coca-cola zero coffeinfree 1,5l ew pet flasche","coca-cola zero coffeinfree 1,5l ew pet flasche"
37,OOHReceipt_20200110_142136,bifi roll xxl,,bifi,bifi roll xxl 75gr 1er,bifi roll xxl 75gr 1er
39,OOHReceipt_20200110_142136,coke zero 1 l pet,,cola,"coca-cola zero coffeinfree 1,5l ew pet flasche","coca-cola zero coffeinfree 1,5l ew pet flasche"
42,OHHReceipt_20200110_125936,cafe creme m,creme,cafe,minges cafe creme dark roast 100 +8 pads 756 gr btl,minges cafe creme dark roast 100 +8 pads 756 gr btl
45,OOHReceipt_20200110_141131,coke zero 1 l pet,,cola,"coca-cola zero coffeinfree 1,5l ew pet flasche","coca-cola zero coffeinfree 1,5l ew pet flasche"
47,OOHReceipt_20200114_101437,"sprite 0,2",,sprite,"sprite 2,0l ew pet flasche","sprite 6x2,0l ew pet flasche"
52,OHHReceipt_20200110_140838,coke zero 1 l pet,,cola,"coca-cola zero coffeinfree 1,5l ew pet flasche","coca-cola zero coffeinfree 1,5l ew pet flasche"
67,OOHReceipt_20200114_091101,"rotbier 0,4",rotbier,BIER,rotbier,"veldensteiner rotbier 0,5 ltr buegelverschlussflasche"
68,OOHReceipt_20200114_200422,kürbiskern-brötchen,kuerbiskern,broetchen,slooow bio organic kuerbiskern broetchen 320gr 4 stueck,slooow bio organic kuerbiskern broetchen 320gr 4 stueck
70,OOHReceipt_20200114_084342,doppelpack mc menue,,doppelpack,airwick fresh matic cotton+weisser flieder 2x 250ml doppelpack aerosol,merci finest selection doppelpack einmal 2x danke sagen 2x250g


### Generate file for Fiver Review

In [88]:
df_fiver_review = df_outofhome_ok[['client_receipt_id','receipt_item_description','essence_in_item']]
df_fiver_review = df_fiver_review[df_fiver_review['receipt_item_description'].isna()==False]
df_fiver_review['receipt_item_description'] = df_outofhome['receipt_item_description'].str.replace('\d+', '')
df_fiver_review = df_fiver_review[df_fiver_review['essence_in_item'].isna()==False]
df_fiver_review = pd.DataFrame(df_fiver_review.groupby(['receipt_item_description','essence_in_item']).first())
df_fiver_review['essence have meaning?'] = ''
df_fiver_review['essence correct?'] = ''
df_fiver_review.sort_values('essence_in_item', ascending=True,inplace=True)


# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('essence_review.xlsx', engine='xlsxwriter')
df_fiver_review.to_excel(writer, sheet_name='essence_review',index=True)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

print(len(df_fiver_review))
df_fiver_review
dfunique_essence

680


array(['chicken tikka', 'cola', nan, 'pasta', 'cafe', 'bifi',
       'milchschnitte', 'fladen', 'kaiserbroetchen', 'quarkbaellchen',
       'mix', 'klopfer', 'aperitif', 'salat-croutons', 'bier', 'radler',
       'schinkennudeln', 'kichererbsen', 'mohnschnecken', 'doppelpack',
       'gemuese', 'milchkaffee', 'mozzarella', 'sprite', 'mineralwasser',
       'pizza', 'suppe', 'salzstange', 'laugenbrezen', 'espresso',
       'macchiato', 'gelbwurst', 'kartoffelsalat', 'schnitzel',
       'schnittlauch', 'limonade', 'latte macchiato', 'hot dog',
       'handstueck', 'samba', 'teehaus', 'kamillentee', 'kaese',
       'trauben', 'broetchen', 'salzstangen', 'selters', 'bionade',
       'cheddar', 'apfeltaschen', 'semmeln', 'backwaren', 'espresso-nero',
       'milchbroetchen', 'laugenstange', 'erdbeersahne', 'krapfen',
       'schafskaese', 'sprizz', 'erdnuesse', 'jever', 'nacs', 'calamari',
       'akropolis', 'weissbier', 'gyros', 'teller', 'tomatensuppe',
       'oktopusarme', 'apfelschorl

In [1]:
from pandas_profiling import ProfileReport

ModuleNotFoundError: No module named 'pandas_profiling'

In [90]:
df_outofhome_ok.describe()

,purchase_total,store_phone,receipt_item_code,receipt_item_price,receipt_item_quantity,identified_gtin,product_confidence
count,1319.000000,1.082000e+03,2.580000e+02,1449.000000,1480.000000,5.800000e+01,92.000000
mean,22.792866,1.531253e+10,6.452930e+10,7.464617,1.428505,4.230679e+12,0.862565
std,53.141368,3.632601e+10,5.147955e+11,56.636205,2.739406,1.619352e+12,0.166772
min,0.340000,1.000021e+07,3.000000e+00,-10.000000,0.082000,4.221194e+07,0.343000
25%,3.780000,9.131456e+08,5.733775e+04,0.990000,1.000000,4.008444e+12,0.869750
50%,8.130000,9.113006e+09,8.676600e+04,2.200000,1.000000,4.043040e+12,0.950000
75%,18.850000,9.123942e+09,1.000010e+05,4.400000,1.000000,4.310030e+12,0.950000
max,410.000000,4.991518e+11,4.305616e+12,959.950000,48.000000,9.000332e+12,0.990000
